In [12]:
import pandas as pd
import os 

def read_data(file_path, sheet_name, save_path):

    # Read the specific sheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    ## drop unnecessary columns 

    columns_to_drop = ['Mouse_ID', 'Tumor_Side', 'Age']
    df = df.drop(columns=columns_to_drop)

    # Specify the treatment type you want to filter by
    treatment_type = [0,1,11,10]  

    # Filter the DataFrame based on the treatment column
    sham_df = df[df['Treatment'] == treatment_type[0]]
    sham_df = sham_df.drop(columns= 'Treatment')

    ir_df = df[df['Treatment'] == treatment_type[1]]
    ir_df = ir_df.drop(columns= 'Treatment')

    aspirin_df = df[df['Treatment'] == treatment_type[2]]
    aspirin_df = aspirin_df.drop(columns= 'Treatment')

    ir_aspirin_df = df[df['Treatment'] == treatment_type[3]]
    ir_aspirin_df = ir_aspirin_df.drop(columns= 'Treatment')
    # Show the filtered DataFrame
    # print(sham_df.head())

    # Save column names to a separate file
    columns_path = os.path.join(save_path, "columns.txt")
    with open(columns_path, 'w') as f:
        f.write(','.join(df.columns))

    return sham_df, ir_df, aspirin_df, ir_aspirin_df


def combine_dfs(sham_df, ir_df, aspirin_df, ir_aspirin_df, save_path):
    # Store DataFrames in a dictionary for better management
    dfs = {
        'sham': sham_df,
        'ir': ir_df,
        'aspirin': aspirin_df,
        'ir+aspirin': ir_aspirin_df
    }

    # Empty list to store results
    merged_dataframes = []

    # Iterate over pairs of DataFrame names and DataFrames
    for name1, df1 in dfs.items():
        for name2, df2 in dfs.items():
            if name1 < name2:  # This check avoids repeating pairs and self-merging
                # Add a new column to each DataFrame to indicate the origin (0 or 1)
                df1_modified = df1.copy()
                df1_modified['Origin'] = 0  # 0 for rows from the first DataFrame
                df1_modified = df1_modified[['Origin'] + [col for col in df1_modified.columns if col != 'Origin']]
                
                df2_modified = df2.copy()
                df2_modified['Origin'] = 1  # 1 for rows from the second DataFrame
                df2_modified = df2_modified[['Origin'] + [col for col in df2_modified.columns if col != 'Origin']]
                
                # Merge the pair of modified DataFrames
                merged_df = pd.concat([df1_modified, df2_modified])
                
                # Store the merged DataFrame with a descriptive name
                merged_dataframes.append((f"{name1}_{name2}", merged_df))

    # Output the results and optionally save them
    for name, df in merged_dataframes:
        # Save DataFrame to CSV without headers
        full_path = os.path.join(save_path, f"{name}.csv")
        df.to_csv(full_path, index=False, header=False)
        print(f"Saved DataFrame '{name}' to '{full_path}'")




# Load the Excel file
file_path = '.\\raw_data\Final_Spreadsheet_separated_by_age.xlsx'
sheet_name = 'Age_W5'  # Replace with your actual sheet name
save_path = '.\\data\output_preprocess'
## creating the source dfs for sham, IR, Aspirin and IR+Aspirin. 
sham_df, ir_df, aspirin_df, ir_aspirin_df = read_data(file_path, sheet_name, save_path)

# Usage example:
combine_dfs(sham_df, ir_df, aspirin_df, ir_aspirin_df, save_path)


Saved DataFrame 'ir_sham' to '.\data\output_preprocess\ir_sham.csv'
Saved DataFrame 'ir_ir+aspirin' to '.\data\output_preprocess\ir_ir+aspirin.csv'
Saved DataFrame 'aspirin_sham' to '.\data\output_preprocess\aspirin_sham.csv'
Saved DataFrame 'aspirin_ir' to '.\data\output_preprocess\aspirin_ir.csv'
Saved DataFrame 'aspirin_ir+aspirin' to '.\data\output_preprocess\aspirin_ir+aspirin.csv'
Saved DataFrame 'ir+aspirin_sham' to '.\data\output_preprocess\ir+aspirin_sham.csv'


In [24]:
print(len(sham_df),len(ir_df), len(aspirin_df), len(ir_aspirin_df))
print(len(sham_df.columns))

12 10 10 10
29


## adding the names of columns to output of r code 

In [15]:
import pandas as pd

# Path to the .txt file containing names
names_file_path = 'C:\\Shabani\\Projects\\tumor_latency\data\\output_preprocess\\columns.txt'
# Path to the directory containing the CSV files
csv_directory_path = 'C:\\Shabani\\Projects\\tumor_latency\\data\\output_r\\'

# Read names from the .txt file into a list
with open(names_file_path, 'r') as file:
    names = file.read().splitlines()

# Read CSV files
csv_files = [f for f in os.listdir(csv_directory_path) if f.endswith('.csv')]

# Read the first CSV file to modify
first_table = pd.read_csv(csv_directory_path + csv_files[0])

# Map the 'V1' column to names using the list from the .txt file
# Ensure that the indexes in 'V1' are valid indexes in the names list
first_table['Names'] = first_table['V1'].apply(lambda x: names[x-1] if x <= len(names) else 'Index out of range')

# Save or display the modified table
print(first_table)  # Print the DataFrame
# Optionally, save back to csv
first_table.to_csv(csv_directory_path + 'modified_' + csv_files[0], index=False)


    V1          V2         V3          V4          V5             V6  \
0    2  -14.103498   0.758340  -15.589817  -12.617179     345.880202   
1    3   18.291639   0.581321   17.152272   19.431007     990.087961   
2    4 -506.956163  26.637416 -559.164539 -454.747786     362.206818   
3    5   82.266591   1.516651   79.294009   85.239172    2942.223950   
4    9 -248.859346   2.919214 -254.580901 -243.137791    7267.347350   
5   14  102.411078   0.816817  100.810147  104.012009   15719.719058   
6   15   -0.007660   0.417750   -0.826435    0.811115       0.000336   
7   16  -42.858815   0.467386  -43.774874  -41.942755    8408.702062   
8   17    0.037758   0.598427   -1.135137    1.210652       0.003981   
9   20  -13.148926   0.740068  -14.599433  -11.698418     315.672481   
10  21  -74.575038   1.431405  -77.380541  -71.769535    2714.324075   
11  23  115.877230   0.284099  115.320407  116.434053  166363.407834   
12  24   -0.761578  11.644679  -23.584728   22.061573       0.00

In [17]:
import pandas as pd

# Path to the .txt file containing names
names_file_path = 'C:\\Shabani\\Projects\\tumor_latency\\data\\output_preprocess\\columns.txt'
# Path to the directory containing the CSV files
csv_directory_path = 'C:\\Shabani\\Projects\\tumor_latency\\data\\output_r\\'

# Read names from the .txt file into a list
with open(names_file_path, 'r') as file:
    names = file.read().splitlines()

# Read CSV files
csv_files = [f for f in os.listdir(csv_directory_path) if f.endswith('.csv')]

# Read the first CSV file to modify
first_table = pd.read_csv(csv_directory_path + csv_files[0])

# Function to map indices to names
def index_to_name(index):
    # Check if index is an integer and within the valid range
    if isinstance(index, int) and 1 <= index <= len(names):
        return names[index - 1]  # Convert 1-based index to 0-based
    else:
        return 'Index out of range'

# Map the 'V1' column to names
first_table['Names'] = first_table['V1'].apply(index_to_name)

# Save or display the modified table
print(first_table)  # Print the DataFrame
# Optionally, save back to csv
first_table.to_csv(csv_directory_path + 'modified_' + csv_files[0], index=False)


    V1          V2         V3          V4          V5             V6  \
0    2  -14.103498   0.758340  -15.589817  -12.617179     345.880202   
1    3   18.291639   0.581321   17.152272   19.431007     990.087961   
2    4 -506.956163  26.637416 -559.164539 -454.747786     362.206818   
3    5   82.266591   1.516651   79.294009   85.239172    2942.223950   
4    9 -248.859346   2.919214 -254.580901 -243.137791    7267.347350   
5   14  102.411078   0.816817  100.810147  104.012009   15719.719058   
6   15   -0.007660   0.417750   -0.826435    0.811115       0.000336   
7   16  -42.858815   0.467386  -43.774874  -41.942755    8408.702062   
8   17    0.037758   0.598427   -1.135137    1.210652       0.003981   
9   20  -13.148926   0.740068  -14.599433  -11.698418     315.672481   
10  21  -74.575038   1.431405  -77.380541  -71.769535    2714.324075   
11  23  115.877230   0.284099  115.320407  116.434053  166363.407834   
12  24   -0.761578  11.644679  -23.584728   22.061573       0.00